# TYTX - XML Support

TYTX also supports XML serialization, using the same typing logic as JSON.

In [ ]:
from decimal import Decimal
from datetime import date

from genro_tytx import as_xml, as_typed_xml, from_xml

## Data Structure for XML

For XML, TYTX uses a data structure that maps:
- **attrs**: element attributes
- **value**: content (scalar, children, or list)

```python
{"tag": {"attrs": {...}, "value": ...}}
```

## 1. Standard XML vs Typed XML

In [ ]:
# Sample data
product = {
    "product": {
        "attrs": {"id": 123},
        "value": {
            "name": {"attrs": {}, "value": "Laptop"},
            "price": {"attrs": {}, "value": Decimal("999.99")},
            "date": {"attrs": {}, "value": date(2025, 12, 4)},
        }
    }
}

print("Standard XML:")
print(as_xml(product))
print()
print("Typed XML (TYTX):")
print(as_typed_xml(product))

## 2. Parsing Typed XML

In [ ]:
# XML with typed values
xml_typed = '''<product id="123::L">
    <name>Laptop</name>
    <price>999.99::N</price>
    <date>2025-12-04::D</date>
</product>'''

parsed = from_xml(xml_typed)

print("Parsed XML:")
print(f"  Tag: product")
print(f"  ID (attr): {parsed['product']['attrs']['id']} ({type(parsed['product']['attrs']['id']).__name__})")

value = parsed['product']['value']
print(f"  Name: {value['name']['value']}")
print(f"  Price: {value['price']['value']} ({type(value['price']['value']).__name__})")
print(f"  Date: {value['date']['value']} ({type(value['date']['value']).__name__})")

## 3. Repeated Elements (Lists)

In [ ]:
# Invoice with repeated line items
invoice = {
    "invoice": {
        "attrs": {"number": "INV-001"},
        "value": {
            "customer": {"attrs": {}, "value": "Acme Corp"},
            "date": {"attrs": {}, "value": date(2025, 12, 4)},
            "item": [
                {"attrs": {"qty": 2}, "value": {
                    "desc": {"attrs": {}, "value": "Product A"},
                    "price": {"attrs": {}, "value": Decimal("50.00")}
                }},
                {"attrs": {"qty": 1}, "value": {
                    "desc": {"attrs": {}, "value": "Product B"},
                    "price": {"attrs": {}, "value": Decimal("99.99")}
                }}
            ],
            "total": {"attrs": {}, "value": Decimal("199.99")}
        }
    }
}

xml_output = as_typed_xml(invoice)
print("Invoice XML:")
print(xml_output)

In [ ]:
# Parse invoice
parsed_invoice = from_xml(xml_output)

inv = parsed_invoice['invoice']
print(f"Invoice: {inv['attrs']['number']}")
print(f"Date: {inv['value']['date']['value']}")
print(f"Customer: {inv['value']['customer']['value']}")

print("\nLine items:")
items = inv['value']['item']
for item in items:
    qty = item['attrs']['qty']
    desc = item['value']['desc']['value']
    price = item['value']['price']['value']
    print(f"  {qty}x {desc}: {price}")

print(f"\nTotal: {inv['value']['total']['value']}")

## 4. Typed Attributes

In [ ]:
# XML with typed attributes
xml_attrs = '''<order id="12345::L" date="2025-12-04::D" total="299.99::N">
    <status>shipped</status>
</order>'''

parsed = from_xml(xml_attrs)
attrs = parsed['order']['attrs']

print("Typed attributes:")
for name, value in attrs.items():
    print(f"  {name}: {value} ({type(value).__name__})")

## 5. XML Roundtrip

In [ ]:
# Original data
original = {
    "record": {
        "attrs": {},
        "value": {
            "id": {"attrs": {}, "value": 42},
            "amount": {"attrs": {}, "value": Decimal("123.45")},
            "date": {"attrs": {}, "value": date(2025, 1, 15)},
            "active": {"attrs": {}, "value": True}
        }
    }
}

# Serialize
xml_str = as_typed_xml(original)
print("Generated XML:")
print(xml_str)

# Re-parse
restored = from_xml(xml_str)

print("\nRestored values:")
for field, content in restored['record']['value'].items():
    val = content['value']
    print(f"  {field}: {val} ({type(val).__name__})")

## 6. When to Use XML?

| Scenario | Recommended Format |
|----------|--------------------|
| Modern REST APIs | JSON |
| Legacy system integration | XML |
| Electronic invoicing | XML |
| Complex configurations | XML |
| Web data exchange | JSON |

TYTX supports both with the same typing semantics!

## Next Notebook

In the next notebook we'll explore **advanced features**:
- Custom registry
- Custom types
- Pydantic integration